In [19]:
from scipy.io import netcdf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from numpy import dtype
import pandas as pd
import xarray as xr



In [20]:
import os
os.getcwd()

'/Users/chinhthon/Desktop/PM2_5_Data/weather_monthly_trial'

In [21]:
# open a netCDF file to read
filename = "data.nc 2"
ncin = Dataset(filename, 'r' , format='NETCDF4')

In [28]:
print(ncin.variables['time'])

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (300,)
filling on, default _FillValue of -2147483647 used


In [23]:
u10 = ncin.variables['u10'][:]
v10 = ncin.variables['v10'][:]
d2m = ncin.variables['d2m'][:]
t2m = ncin.variables['t2m'][:]
skt = ncin.variables['skt'][:]
sp = ncin.variables['sp'][:]
tp = ncin.variables['tp'][:]
time = ncin.variables['time'][:]
lat = ncin.variables['latitude'][:]
lon = ncin.variables['longitude'][:]

In [24]:
u10.shape

(300, 181, 101)

In [7]:
# check dimensions
print(u10.shape)
print(v10.shape)
print(d2m.shape)
print(t2m.shape)
print(skt.shape)
print(sp.shape)
print(tp.shape)
print(time.shape)
print(lat.shape)
print(lon.shape)

(300, 181, 101)
(300, 181, 101)
(300, 181, 101)
(300, 181, 101)
(300, 181, 101)
(300, 181, 101)
(300, 181, 101)
(300,)
(181,)
(101,)


In [34]:
print(
    max(ncin.variables['latitude'][:]),
    min(ncin.variables['latitude'][:]),
    max(ncin.variables['longitude'][:]),
    min(ncin.variables['longitude'][:])
    )

25.0 7.0 111.0 101.0


In [35]:
ncin.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (300,)
filling on, default _FillValue of -2147483647 used

In [40]:
# convert hrs since 1900 to date-time
start_date = pd.Timestamp('1900-01-01 00:00:00.0')
years = start_date + pd.to_timedelta(time, unit='h')
print(years)

In [42]:

# create an xarray data object
u10_da = xr.DataArray(u10, dims=('time', 'lat', 'lon'),
                       coords={'time': years, 'lat': lat, 'lon': lon})
# convert the object to dataframe
u10_df = u10_da.to_dataset(name='u10')
u10_df

<xarray.Dataset>
Dimensions:  (time: 300, lat: 181, lon: 101)
Coordinates:
  * time     (time) datetime64[ns] 1998-01-01 1998-02-01 ... 2022-12-01
  * lat      (lat) float32 25.0 24.9 24.8 24.7 24.6 24.5 ... 7.4 7.3 7.2 7.1 7.0
  * lon      (lon) float32 101.0 101.1 101.2 101.3 ... 110.7 110.8 110.9 111.0
Data variables:
    u10      (time, lat, lon) float64 1.694 1.804 1.891 1.968 ... nan nan nan

In [47]:
df = u10_df.to_dataframe()
df_reset = df.reset_index()
# Melt the DataFrame to long form
df_long_u10 = df_reset.melt(id_vars=['time', 'lat', 'lon'],
                        var_name='variable', value_name='value')



In [49]:
df_long_u10

,time,lat,lon,variable,value
0,1998-01-01,25.0,101.000000,u10,1.693569
1,1998-01-01,25.0,101.099998,u10,1.804002
2,1998-01-01,25.0,101.199997,u10,1.891213
3,1998-01-01,25.0,101.300003,u10,1.967845
4,1998-01-01,25.0,101.400002,u10,2.010935
...,...,...,...,...,...
5484295,2022-12-01,7.0,110.599998,u10,NaN
5484296,2022-12-01,7.0,110.699997,u10,NaN
5484297,2022-12-01,7.0,110.800003,u10,NaN
5484298,2022-12-01,7.0,110.900002,u10,NaN
